Install Necessary Libarary

In [ ]:
%pip install --quiet --upgrade ipywidgets langchain langchain_openai

```mermaid
graph LR
A[Document] --> |Split| B[Chunks]
    B --> |Create Embedding| C[Vector Store]
D[User] --> E(Query)
    E --> |Create Embedding| F(Similarity Search)
    F --> C
    E --> G[LLMS]
    C --> G
    
```

Import OpenAI API Key

In [ ]:
import os
f = open('openaikey.txt')
os.environ["OPENAI_API_KEY"]=f.read()

Import Document and Split into Chuncks

In [ ]:
from langchain_community.document_loaders.pdf import PyPDFLoader
pyPDFLoader = PyPDFLoader("resources/DriversGuidefinal2014.pdf")
data = pyPDFLoader.load()

Split using Text Splitter

In [ ]:
from langchain_text_splitters import PythonCodeTextSplitter
textSplitter = PythonCodeTextSplitter(chunk_size=1000,chunk_overlap=300)
pages= pyPDFLoader.load_and_split(text_splitter=textSplitter)

Print Relevant information

In [ ]:
print(f"Total Documents Length before text split {len(data)}")
print(f"Total Documents after the text split {len(pages)}")


Initiate Embedding

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
embedding = OpenAIEmbeddings()
db = FAISS.from_documents(documents=pages,embedding=embedding)
print(db.index.ntotal)

Try executing  a query

In [ ]:
query = "What is this document about ? "
result = db.similarity_search(query=query)


Explore results

In [ ]:
print(result[2])

Create a Chat bot

In [ ]:
from langchain.chains.question_answering.chain import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

openAI = ChatOpenAI()

chain = load_qa_chain(openAI,chain_type="stuff")
query = "I am from India can I exchange my Indian licence to get Alberta license ?"
docs = db.similarity_search(query)
response = chain.invoke(input={"input_documents":docs, "question":query})

Explore result

In [ ]:
print(response)

Create a Chat Component

In [ ]:
from IPython.display import display
import ipywidgets as widgets

qa = ConversationalRetrievalChain.from_llm(openAI, db.as_retriever())

chat_history = []

def on_sumbit(_):
    query = input_box.value
    input_box.value=""
    print(chat_history)
    if query.lower() == 'exit':
        print("Thank you for chatting with me")
        return
    
    result = qa.invoke(input={"question":query,"chat_history":chat_history})
    chat_history.append((query, result['answer']))
   
    display(widgets.HTML(f"Number of matches found : {len(result)}"))
    display(widgets.HTML(f"<b>User Query</b> : {query}"))
    display(widgets.HTML(f'<b><font color="blue">Chat Bot :</font></b> {result["answer"]}'))

print("Welcome to the chatbot")
input_box = widgets.Text(placeholder="Enter the Question")
input_box.on_submit(on_sumbit)

display(input_box)

Welcome to the chatbot


/var/folders/df/4n57lpzd4z986szq__z25w840000gn/T/ipykernel_22967/1897808263.py:25: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  input_box.on_submit(on_sumbit)


Text(value='', placeholder='Enter the Question')

[]
None


HTML(value='Number of matches found : 3')

...

HTML(value='<b><font color="blue">Chat Bot :</font></b> To obtain a Class 7 learner\'s driver\'s license, you …

[('What is the minimum number of questions to be correct to get GDL 7 license', "To obtain a Class 7 learner's driver's license, you must score a minimum of 25 correct responses out of 30 questions on the knowledge test.")]
None


HTML(value='Number of matches found : 3')

HTML(value='<b>User Query</b> : what is the wait period after getting GDL 7 until I could apply for GDL 5')

HTML(value='<b><font color="blue">Chat Bot :</font></b> The wait period after obtaining a Class 7 Learner\'s l…